# Content Creator With Reflexion

### Define Pydantic DataModels

In [1]:
from pydantic import BaseModel, Field
from typing import List

In [2]:
class AnswerQuestion(BaseModel):
    """Answer the Question"""
    # thought_process:str = Field(description="Thought process behind the answer. This is between the think tokens in the output of the model. <think>thought_process</think>")
    answer:str = Field(description= "Extremely detailed answer to the question")
    search_queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer")
    missing: str = Field(description="Critique of what information is missing")
    superfluous: str = Field(description="Critique of what is superfluous")
    
    
class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question"""
    references: List[str] = Field(description="Citations motivating your updated answer")


### Define Chains

In [3]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import datetime
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import PydanticToolsParser, JsonOutputToolsParser
from langchain_core.messages import HumanMessage
from langchain.output_parsers import StructuredOutputParser, PydanticOutputParser

In [4]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert content creator who creates interesting, exciting and mind bending content for youtube and instagram.
            Current time: {time}

            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. Identify superflous information
            4. After this, **list 1-3 search queries separately** for researching improvements. Do not include them inside the reflection.

            You are required to provide the output in the following json format
            {output_format}
            """
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Answer the user's question above using the required format"),
    ]
).partial(
    time= lambda: datetime.datetime.now().isoformat(),
)

In [5]:
format_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are helpful assistant who takes the input and structures in the given format
        {input}

        {format_instructions}
        """)
    ]
)

def get_output_format_prompt(pydantic_data_model, format_prompt_template):
    parser = PydanticOutputParser(pydantic_object=pydantic_data_model)
    format_instructions = parser.get_format_instructions()
    format_prompt = format_prompt_template.partial(format_instructions = format_instructions)

    return format_prompt

In [6]:
first_responder_output_instruction = f"""
```json
{AnswerQuestion.model_fields}
```
"""

actor_agent_output_prompt = get_output_format_prompt(AnswerQuestion, format_prompt_template)

In [7]:
revisor_output_instruction = f"""
```json
{ReviseAnswer.model_fields}
```
"""
revisor_agent_output_prompt = get_output_format_prompt(pydantic_data_model=ReviseAnswer, format_prompt_template=format_prompt_template)

In [8]:
# main_llm_model =  "qwen2.5:32b"
main_llm_model =  "qwen3:14b"
sub_llm_model = "llama3.2:3b-instruct-fp16"

llm = ChatOpenAI(
    api_key="ollama",
    model = main_llm_model,
    base_url="http://localhost:11434/v1",
    temperature=0.9,
)

# using smaller model for output format
output_llm = ChatOpenAI(
    api_key="ollama",
    model=sub_llm_model,
    base_url="http://localhost:11434/v1",
    temperature=0
)

In [9]:
first_responder_prompt_template = actor_prompt_template.partial(first_instruction = "Provide a extremely detailed answer", 
                                                                output_format = first_responder_output_instruction)

first_responder_chain = first_responder_prompt_template | llm | actor_agent_output_prompt | output_llm.with_structured_output(schema=AnswerQuestion)

In [10]:
revise_instructions = """Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer.
"""

In [11]:
revisor_chain = actor_prompt_template.partial(first_instruction=revise_instructions,
                                              output_format = revisor_output_instruction) | llm | revisor_agent_output_prompt | output_llm.with_structured_output(schema=ReviseAnswer)

In [12]:
final_content_creator_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """##Personality
            You are an expert content creator who creates interesting, exciting and mind bending content for youtube and instagram from the given content.
            You are given a user's query and answers from different agents and tools as observations which cover different aspects, topics and \
            nuances that are pertaining to the user's query. You are required to create an extremely detailed content from these observations covering all the points \
            which answer's the user questoin and you need to make sure the generated content is for sure going be a viral and reach Millions. 
            You are required to generate the content for the user query from the given observations only.
            """
        ),
        MessagesPlaceholder(variable_name="user_query"),
        MessagesPlaceholder(variable_name="observations"),
        (
            "system",
            "Generate a VIRAL content for YOUTUBE and INSTAGRAM based on the user_query and observations"
        ),
        MessagesPlaceholder(variable_name="enable_thinking", optional=True)
    ]
)

content_creator_chain = final_content_creator_prompt | llm

### Testing the chain

In [15]:
response = first_responder_chain.invoke({
    "messages": [HumanMessage("instagram influencers earning more than software developers")]
})
print(response.model_dump_json(indent=4))

{
    "answer": "Instagram influencers can sometimes earn more than software developers due to a combination of factors related to income models, market dynamics, scalability, and visibility. Here\\u2019s a detailed breakdown:\\n\\n1. **Diverse Revenue Streams for Influencers**: Influencers typically monetize through brand partnerships, sponsored content, affiliate marketing, merchandise sales, and subscription-based models (e.g., Patreon). For example, a micro-influencer with 100,000 followers might earn $10\\u2013$50 per post, while a macro-influencer with millions of followers can command six figures per deal. Additionally, influencers often leverage their personal brand to launch products or services, which can generate passive income once established.\\n\\n2. **Scalability and Global Reach**: Content created by influencers can reach millions of people with minimal incremental cost. A single video or post can be repurposed across platforms (Instagram, TikTok, YouTube), maximizing e

In [17]:
response2 = first_responder_chain.invoke({
    "messages": [HumanMessage("AI taking over the content creation")]
})
print(response.model_dump_json(indent=4))

{
    "thought_process": "To address the question of Instagram influencers potentially earning more than software developers, I first need to analyze the economic dynamics of both professions. Software developers typically have stable, high salaries (e.g., $100k–$150k in the U.S.) with long-term career growth and benefits like equity or remote work. Influencers’ earnings are variable: top-tier creators (e.g., Kylie Jenner, MrBeast) can earn millions via brand deals, ads, and product sales, but the median income is far lower, often dependent on engagement rates, niche markets, and platform algorithms. The comparison hinges on outliers versus averages, short-term vs. long-term stability, and sector trends (e.g., AI disrupting tech jobs). I must balance these factors while avoiding outdated data or oversimplifications.",
    "answer": "Instagram influencers and software developers represent two distinct high-earning careers, but their income structures and risks differ drastically. Top in

In [14]:
response = first_responder_chain.invoke({
    "messages": [HumanMessage("write a story to kids telling about Lord RAM")]
})
print(response.model_dump_json(indent=4))

{
    "answer": "The story of Lord Rama, a great king from Hindu mythology, has been retold and reinterpreted in various ways over the centuries. One of the most famous versions is the Ramayana, which tells the story of Rama's journey to rescue his wife Sita from the clutches of the demon king Ravana. The story begins with Rama's birth as a prince, raised by his stepmother Queen Kaikeyi and her husband King Dasharatha. However, Rama is actually the eldest son of King Dasharatha and his first wife, Queen Kaushalya. When Rama reaches adulthood, he is banished from the kingdom for 14 years due to a promise made by Queen Kaikeyi. This event sets off a chain reaction that leads to Rama's journey into the forest with his loyal brother Lakshman and his beloved wife Sita. In the forest, they face numerous challenges and dangers, but ultimately find solace in their love for each other and their devotion to dharma (duty). Meanwhile, back in the kingdom, Queen Kaikeyi's jealousy and ambition driv

In [17]:
response = first_responder_chain.invoke({
    "messages": [HumanMessage("Maintaining a healthy lifestyle in bangalore")]
})
print(response.model_dump_json(indent=4))

{
    "thought_process": "To address the query on maintaining a healthy lifestyle in Bangalore, I began by analyzing the city's unique environmental and cultural factors, such as its climate, urbanization, and tech-driven lifestyle. I structured the answer around key pillars: diet, exercise, mental health, and community engagement. I considered local challenges like air quality during summer and the need for cost-effective solutions. I also reflected on how to balance actionable advice with practicality for diverse demographics in Bangalore. The critique focused on avoiding overly generic suggestions and ensuring actionable, location-specific strategies.",
    "answer": "Maintaining a healthy lifestyle in Bangalore requires adapting to the city's unique dynamics. First, diet should prioritize fresh, seasonal produce like millets, jackfruit, and arecanuts, which thrive in South India. Incorporate local markets for organic groceries and avoid overconsumption of processed foods, a common 

In [22]:
print(response.model_dump_json(indent=4))

{
    "thought_process": "To address the question of Instagram influencers potentially earning more than software developers, I first need to analyze the economic dynamics of both professions. Software developers typically have stable, high salaries (e.g., $100k–$150k in the U.S.) with long-term career growth and benefits like equity or remote work. Influencers’ earnings are variable: top-tier creators (e.g., Kylie Jenner, MrBeast) can earn millions via brand deals, ads, and product sales, but the median income is far lower, often dependent on engagement rates, niche markets, and platform algorithms. The comparison hinges on outliers versus averages, short-term vs. long-term stability, and sector trends (e.g., AI disrupting tech jobs). I must balance these factors while avoiding outdated data or oversimplifications.",
    "answer": "Instagram influencers and software developers represent two distinct high-earning careers, but their income structures and risks differ drastically. Top in

In [24]:
print(response2.model_dump_json(indent=4))

{
    "thought_process": "The user asked about AI taking over content creation for YouTube and Instagram. I need to address current trends, tools, and implications. Start with AI's role in generating text, images, and videos, then discuss efficiency, ethical concerns, and human-AI collaboration. Highlight case studies (e.g., AI-generated viral content) and industry shifts. Ensure the critique identifies gaps like lack of quantitative data and potential oversimplification of ethical debates.",
    "answer": "AI is revolutionizing content creation by automating tasks once reserved for humans. Platforms like YouTube and Instagram now see AI-generated videos, images, and scripts at unprecedented speeds. Tools such as MidJourney, DALL-E 3, and Runway ML enable creators to produce high-quality visuals and edits in seconds, while AI scriptwriters (e.g., Jasper, Copy.ai) generate engaging captions and storylines. Brands like Netflix and TikTok creators use AI to analyze audience preferences an

### Tools

In [13]:
from dotenv import load_dotenv
import os
import json
from typing import List, Dict, Any
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import StructuredTool
from langchain.tools import tool
from langgraph.prebuilt import ToolNode

In [14]:
# Load environment variables from .env file
load_dotenv()

# Access the Tavily API key
tavily_api_key = os.getenv("TAVILY_API_KEY")

if tavily_api_key:
    print("Tavily API Key loaded successfully.")
else:
    print("Tavily API Key not found. Please check your .env file.")

Tavily API Key loaded successfully.


In [15]:
from langchain_core.tools import StructuredTool

from langgraph.prebuilt import ToolNode

tavily_tool = TavilySearchResults(max_results=3)
@tool
def run_queries(search_queries: list[str]):
    """Run the generated queries."""
    return tavily_tool.batch([{"query": query} for query in search_queries])

### Graph

In [16]:
from typing import List, Union
from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph, StateGraph, START
from typing import Literal, Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import ToolCall
import uuid

In [17]:
MAX_ITERATIONS = 1

class State(TypedDict):
    user_query: HumanMessage
    observations: List[AIMessage]
    messages: Annotated[list, add_messages]
    revise_answer_iterations: int
    

def responder_node(state:dict) -> dict:
    response = first_responder_chain.invoke({'messages': state['messages']})
    search_tool_call = ToolCall(id = str(uuid.uuid4()),
                                name=response.__class__.__name__, 
                                args = {"queries":response.search_queries})
    
    ai_message = AIMessage(content=str(response.model_dump_json(exclude = 'search_queries', indent=4)), 
                           tool_calls=[search_tool_call])

    print(f"This is the response of the Responder Node :: \n {ai_message}")
    
    return {"user_query": state['messages'],
            "observations": [AIMessage(content = "Responder Agent:: " + response.model_dump_json(include = 'answer'))],
            "messages": state['messages'] + [ai_message],
            "revise_answer_iterations": 0}

def revisor_node(state:dict)-> dict:
    
    response = revisor_chain.invoke({'messages': state['messages']})
    search_tool_call = ToolCall(id = str(uuid.uuid4()),
                                name=response.__class__.__name__, 
                                args = {"queries":response.search_queries})
    
    ai_message = AIMessage(content=str(response.model_dump_json(exclude = 'search_queries', indent=4)), 
                           tool_calls=[search_tool_call])

    print(f"This is the response of the Revisor Node :: \n {ai_message}")
    
    return {"user_query": state['user_query'],
            "observations": state["observations"] + [AIMessage(content = "Revisor Agent:: " + response.model_dump_json(include = 'answer'))],
            "messages": state['messages'] + [ai_message],
            "revise_answer_iterations": state['revise_answer_iterations']}

def execute_tools(state: dict) -> dict:
    messages = state['messages']
    last_ai_message: AIMessage = messages[-1]

    if not hasattr(last_ai_message, "tool_calls") or not last_ai_message.tool_calls:
        return []

    tool_messages = []
    tool_observations = []
    for tool_call in last_ai_message.tool_calls:
        if tool_call["name"] in ["AnswerQuestion", "ReviseAnswer"]:
            if tool_call["name"] == 'ReviseAnswer':
                print(f"ToolCallName :: {tool_call["name"]}")
                iterations = state['revise_answer_iterations'] + 1
            else:
                print(f"ToolCallName :: {tool_call["name"]}")
                iterations = state['revise_answer_iterations']
            
            call_id = tool_call["id"]
            search_queries = tool_call["args"].get("queries", [])
            query_results_lst = run_queries.invoke({'search_queries': search_queries})
            query_results = {query:query_result for query,query_result in zip(search_queries, query_results_lst)}
            
            # print(f"toolName :: {tool_call['name']}\ncallId :: {tool_call['id']}\nsearch_queries :: {search_queries}")
            tool_messages.append(
                ToolMessage(content=json.dumps(query_results),
                            tool_call_id = call_id)
            )
            tool_observations.append(json.dumps(query_results))

    tool_observations = '\n'.join(tool_observations)
    
    return {"user_query": state['user_query'],
            "observations": state["observations"] + [AIMessage(content = "Tool Executor Agent:: " + tool_observations)],
            "messages" : messages + tool_messages, 
            "revise_answer_iterations": iterations,
           }

def content_creator_node(state:dict)->dict:
    user_query = state['user_query']
    observations = state['observations']

    fnl_content = content_creator_chain.invoke({
    'user_query' : user_query,
    'observations': observations,
    'enable_thinking': ['/no_think']
    })

    return {"user_query": state['user_query'],
            "observations": state["observations"],
            "messages" : state['messages'] + [fnl_content],
            "revise_answer_iterations": state["revise_answer_iterations"],
           }
    

def event_loop(state: dict) -> Literal["content_creator", "execute_tools"]:
    num_iterations = state['revise_answer_iterations']
    if num_iterations >= MAX_ITERATIONS:
        return "content_creator"
    else:
        return "execute_tools"


In [18]:
graph = StateGraph(State)

graph.add_node('draft', responder_node)
graph.add_node("execute_tools", execute_tools)
graph.add_node("revisor", revisor_node)
graph.add_node("content_creator", content_creator_node)



graph.add_edge(START, 'draft')
graph.add_edge("draft", 'execute_tools')
# graph.add_edge("draft", END)

graph.add_edge('execute_tools', "revisor")
graph.add_conditional_edges('revisor', event_loop)
graph.add_edge("content_creator", END)
app = graph.compile()


In [19]:
print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	draft(draft)
	execute_tools(execute_tools)
	revisor(revisor)
	content_creator(content_creator)
	__end__([<p>__end__</p>]):::last
	__start__ --> draft;
	content_creator --> __end__;
	draft --> execute_tools;
	execute_tools --> revisor;
	revisor -.-> content_creator;
	revisor -.-> execute_tools;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	draft(draft)
	execute_tools(execute_tools)
	revisor(revisor)
	content_creator(content_creator)
	__end__([<p>__end__</p>]):::last
	__start__ --> draft;
	content_creator --> __end__;
	draft --> execute_tools;
	execute_tools --> revisor;
	revisor -.-> content_creator;
	revisor -.-> execute_tools;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc
```

In [20]:
response = app.invoke({
    "messages": [HumanMessage("what are the ways to maintain a healthy lifestyle in bangalore for corporate professionals")]
}
)

This is the response of the Responder Node :: 
 content='{\n    "answer": "Maintaining a healthy lifestyle for corporate professionals in Bangalore requires integrating physical, mental, and environmental considerations into a busy schedule. Here’s a detailed approach: 1. **Nutrition**: Opt for meal prepping with local ingredients from organic markets like Koramangala or Indiranagar. Avoid over-reliance on processed corporate cafeteria food. Incorporate South Indian healthy staples like dosa (without ghee), upma, and lentils. Use delivery services like HungerStation or Farm to Table for fresh, balanced meals. 2. **Exercise**: Leverage Bangalore’s green spaces: Jog or cycle in Cubbon Park, Nandi Hills, or along the Bellandur Lake (avoid during pollution peaks). Join yoga studios like YogaWorks or CorePower Yoga for structured sessions. Utilize workplace gym facilities or apps like Nike Training Club for at-home workouts during commutes. 3. **Mental Health**: Address stress with mindfuln

In [21]:
print(response['messages'][-1].content)

<think>

</think>

**🎥 VIRAL YOUTUBE VIDEO SCRIPT: "How to Stay Healthy as a Corporate Professional in Bangalore in 2025" 📍**  
**🎯 FOR YouTube / Instagram Reels / Shorts**  
**⏱️ Duration: 6-7 minutes**  
**🎵 BGM: Upbeat Indie Pop / Motivational Beat**  
**✨ Hashtags: #HealthInBengaluru #WorkLifeBalance #CorporateWellness #BengaluruLife #HealthyProfessionals #MindAndBodyBalance2025**

---

**[OPENING SCENE – 0:00 to 0:15]**  
**🎵 Music starts with a high-energy beat.**  
**🎬 Quick shots: Busy corporate professionals in Bengaluru, traffic, office setups.**  
**🎤 VOICEOVER / ON-SCREEN TEXT:**  
_"Working in Corporate Bangalore? You’re not alone. But staying healthy in a fast-paced city like this is a real challenge. Let’s crack the code on what it takes to live a healthy, balanced life in 2025."_

---

**[SECTION 1: NUTRITION – 0:16 to 1:30]**  
**🎬 Footage of healthy meals, organic stores like Suggi Naturals, and meal delivery services.**  
**🎤 VOICEOVER (exciting tone):**  
_"Let’s st